In [25]:
import pandas as pd
import numpy as np
import re
import string
from nltk.stem import PorterStemmer
ps = PorterStemmer()
import pickle

In [26]:
text="Great Product I like It !"


In [27]:
with open('../static/model/corpora/stopwords/english', 'r') as file:
    sw = file.read().splitlines()

In [28]:

with open('../static/model/model.pickle','rb')as file:
    model=pickle.load(file)

In [29]:
vocab=pd.read_csv('../static/model/vocabulary.txt',header=None)
tokens=vocab[0].tolist()

In [30]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text



In [31]:
def preprocessing(text):
    data=pd.DataFrame([text],columns=['tweet'])
    data['tweet']=data['tweet'].apply(lambda x:" ".join(x.lower() for x in x.split()))
    data["tweet"] = data['tweet'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["tweet"] = data["tweet"].apply(remove_punctuations)
    data["tweet"] = data['tweet'].str.replace('\d+', '', regex=True)
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["tweet"] = data["tweet"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data['tweet']

In [32]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new

In [33]:
preprocessed_text=preprocessing(text)

In [34]:
vectorized_list=vectorizer(preprocessed_text,tokens)

In [35]:
model.predict(vectorized_list)

array([1], dtype=int64)

In [51]:
new_text="I Really Love this product and awsome"

In [52]:
prep_text=preprocessing(new_text)
vected=vectorizer(prep_text,tokens)

In [53]:
model.predict(vected)

array([0], dtype=int64)

In [54]:
def prediction(vected_text):
    result=model.predict(vected_text)
    if result==1:
        return "Negative Feed"
    else:
        return "Positive Feed"


In [55]:
prediction(vected)

'Positive Feed'